In [15]:
# Author: LEITE, G. B. S.
# Description: Applying the TOPSIS method to a decision matrix

In [14]:
import pandas as pd
import numpy as np
from operator import itemgetter

In [21]:
# Decision matrix definition
m = [[1.2, 25, 8, 58000, 7, 5, 1126],
     [1.65, 40, 7, 31000, 7, 8, 334],
     [1.27, 20, 9, 45000, 9, 6, 227],
     [1.77, 25, 7, 50000, 8, 8, 550]]
print(f'Decision matrix: {m}')

# Criterion status definition, so that: 0 corresponds to minimization criteria and 1 to maximization criteria
st = [0, 0, 1, 1, 1, 1, 0]
print(f'Criteria status: {st}')

# Definition of weights for each criterion
pesos = [0.18, 0.2, 0.1, 0.2, 0.12, 0.05, 0.15]
print(f'Criteria weights: {pesos}')

Decision matrix: [[1.2, 25, 8, 58000, 7, 5, 1126], [1.65, 40, 7, 31000, 7, 8, 334], [1.27, 20, 9, 45000, 9, 6, 227], [1.77, 25, 7, 50000, 8, 8, 550]]
Criteria status: [0, 0, 1, 1, 1, 1, 0]
Criteria weights: [0.18, 0.2, 0.1, 0.2, 0.12, 0.05, 0.15]


In [17]:
# As the first step to apply the TOPSIS, in a decision matrix is to normalize it, the normalization function is defined
# Here the Euclidean normalization will be used, as it is the most used in TOPSIS

def normalizar_euclidiana(matriz, status):
  """
  -> Function to calculate the Euclidean normalization of a decision matrix.

   :param matriz: decision matrix.
   :param status: list containing the status of each criterion - if it is a maxization criterion, the list receives 1 and, if it is a minization criterion, the list receives 0.
   :return: transposed normalized matrix.

  """
  
  l = []
  for j in range(len(matriz[0])):
    linha = []
    for i in range(len(matriz)):
      linha.append(matriz[i][j])
    l.append(linha)

  q = []
  for linha in l:
    a = 0
    for item in linha:
      a += (item**2)
    b = (a**(1/2))
    q.append(b)
  
  m_final = []
  for i in range(0, len(status)):
    lin = []
    if status[i] == 0:
      for j in range(0, len(l[i])):
        lin.append(1-(l[i][j]/q[i]))
      m_final.append(lin[:])
    
    else:
      for j in range(0, len(l[i])):
        lin.append(l[i][j]/q[i])
      m_final.append(lin[:])
      
  return m_final

In [18]:
# Function application
m_normec = normalizar_euclidiana(m, st)
print(f'Transposed normalized matrix:\n{np.array(m_normec)}')

Transposed normalized matrix:
[[0.59794652 0.44717646 0.5744934  0.40697111]
 [0.56147099 0.29835358 0.64917679 0.56147099]
 [0.51320024 0.44905021 0.57735027 0.44905021]
 [0.61653304 0.32952628 0.4783446  0.531494  ]
 [0.44905021 0.44905021 0.57735027 0.51320024]
 [0.36369648 0.58191437 0.43643578 0.58191437]
 [0.1447731  0.74631813 0.82758747 0.58226039]]


###**Método Technique for Order Preference by Similarity to Ideal Solution (TOPSIS)**



Let the matrix be normalized (euclidean normalization): $X =
 \begin{pmatrix}
  x_{1,1} & x_{1,2} & \cdots & x_{1,j} \\
  x_{2,1} & x_{2,2} & \cdots & x_{2,j} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  x_{i,1} & x_{i,2} & \cdots & x_{i,j}
 \end{pmatrix}$;

 and the weight vector: $w =
 \begin{pmatrix}
  w_{1} & w_{2} & \cdots & w_{k} \\
 \end{pmatrix}$, so that $\sum_{j=i}^k w_{j} = 1$.

 First, the normalized matrix ($T$) weighted by the weights $w$ is calculated:

 $T =
 \begin{pmatrix}
  t_{1,1} & t_{1,2} & \cdots & t_{1,j} \\
  t_{2,1} & t_{2,2} & \cdots & t_{2,j} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  t_{i,1} & t_{i,2} & \cdots & t_{i,j}
 \end{pmatrix}$, onde $t_{ij} = x_{ij}w{j}$

Then, a positive ideal alternative $A^+ =
 \begin{pmatrix}
  t^+_{1} & t^+_{2} & \cdots & t^+_{n} \\
 \end{pmatrix}$ and a negative ideal alternative $A^- =
 \begin{pmatrix}
  t^-_{1} & t^-_{2} & \cdots & t^-_{n} \\
 \end{pmatrix}$  are defined, where:

$t^+_{i} = max(t_{ij})$, if the criterion is to maximize, and $min(t_{ij})$, if the criterion is to minimize;

$t^-_{i} = min(t_{ij})$, if the criterion is to maximize, and $max(t_{ij})$, if the criterion is to minimize.

It is important to be aware that the normalization method corrects the direction of the criteria. In the case of the function presented in this code, as the direction is corrected, when selecting the ideal alternatives, it is assumed that all criteria become maximization.

Now, the distances $d^+_{i}$ and $d^-_{i}$ of each alternative to the ideal alternatives are calculated:

$d^+_{i} = sqrt(\sum_{j=i}^n (t^+_{j} - t_{ij})^2)$ and $d^-_{i} = sqrt(\sum_{j=i}^n (t^-_{j} - t_{ij})^2)$.

The value of each alternative is given by: $r_{i} = \frac{d^-_{i}}{d^+_{i} + d^-_{i}}$.

The alternatives are ordered according to the ascending order of the vector $r = \begin{pmatrix}
r_{1} & r_{2} & \cdots & r_{i} \\
\end{pmatrix}$.

In [19]:
def topsis(matriz, pesos):
  """
   -> Function to evaluate the alternatives from the Technique for Order Preference by Similarity to Ideal Solution (TOPSIS) method.

   :param matriz: transposed normalized decision matrix.
   :param pes: list containing the weights of each criterion analyzed.
  
   The function returns a print with the classification of the evaluated alternatives.

   """

  c = 0
  m_p = []

  while c < len(pesos):                         # The decision matrix is weighted by the weights of their respective criteria
    col = []
    for i in matriz[c]:
      col.append(i*pesos[c])
    m_p.append(col[:])
    col.clear()
    c += 1

  a_positivo = []
  a_negativo = []
  for i in m_p:                                 # Positive and negative ideal alternatives are generated
    a_positivo.append(max(i))
    a_negativo.append(min(i))
  
  matrizt = []                                  # The weighted matrix - which was already transposed - is transposed
  for j in range(len(m_p[0])):
    linha = []
    for i in range(len(m_p)):
      linha.append(m_p[i][j])
    matrizt.append(linha)
  
  dic = {}                                       # A dictionary with the weighted matrix is created
  for i in range(0, len(m_p[0])):
    dic[i] = matrizt[i]

  d_positivo = []
  d_negativo = []
  for i in range(0, len(dic)):                   # Calculate the positive and negative d
    dmais = dmenos = 0
    for j in range(0, len(a_positivo)):
      dmais += (a_positivo[j] - dic[i][j])**2
      dmenos += (a_negativo[j] - dic[i][j])**2
    d_positivo.append(dmais**(1/2))
    d_negativo.append(dmenos**(1/2))

  score = []                                     # The score for each alternative is calculated
  for i in range(0, len(matriz[0])):
    pont = d_negativo[i]/(d_negativo[i] + d_positivo[i])
    score.append(pont)
  
  dics = {}                                      # A dictionary with the score is also created
  for i in range(0, len(score)):
    dics[i] = score[i]

  print('='*26, end=' ')
  print('\033[91m'+"Ranking TOPSIS"+'\033[0m', end=' ')
  print('='*26)

  ranking = sorted(dics.items(), key=itemgetter(1), reverse=True)         # And, finally, the function returns the result with the classification of alternatives
  for i, v in enumerate(ranking):
    print(f'{i+1}st place: Alternative {v[0]+1} with a {v[1]} point rating')

In [20]:
# Application of the TOPSIS method in the normalized matrix
topsis(m_normec, pesos)

========================== Ranking TOPSIS ==========================
1st place: Alternative 3 with a 0.8212780863451156 point rating
2st place: Alternative 4 with a 0.619337770745421 point rating
3st place: Alternative 2 with a 0.48326275721711354 point rating
4st place: Alternative 1 with a 0.44653041228768514 point rating
